In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os


In [ ]:
#Load data into using Pandas Read_csv
dataset_train = pd.read_csv("google-stock-price-data\Google_Stock_Price_Train.csv")
dataset_train.head()

In [ ]:
#Lets understand how Dataframe.iloc is working
# Iloc representd for "Index Location of" 
# Try below commands removing comments
#dataset_train.iloc[:]                       # This will give you all dataframe values
#dataset_train.iloc[1]                        # this will return you your first index data#
#dataset_train.iloc[:,0]                    # return all data of om index column
#type(dataset_train.iloc[:,1])           # will return you all data of column 1 But output will be in Pandas.core.series.Series
#dataset_train.iloc[:,1:2]             # Prefered  # will return all data of column 1 But output will be in Pandas Dataframe

# Get your input data in variable (Open column data is our input data for our model)
trainset = dataset_train.iloc[:,1:2].values        # .values return array 
trainset

In [ ]:
# Now we will scale out trainset AKA input value within scale of 0 to 1 
# Min Max Scaler is precedure to Scale original value within maped value without loosing original information
# This is Normalization technique
#Normalization is used to scale the data between 0 and 1. It is defined as
#Yi = [Xi - min(X)]/[max(X) - min(X)]
#Where Xi is the i th data point and min represents the minimum and Maximum represents maximum. So Xi converts to Yi

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))        # Feature_range you can define for your output range (default (0,1))
training_set_scaled = sc.fit_transform(trainset)

In [ ]:
training_set_scaled                                

#### Create our training and testing dataset By taking first 60 days data to predict 61st days stock price
#### split data in x_train and y_train 

In [ ]:
# now we will update our dataset in a way that our neural network will learn on basis of first two month data(60 days) to 
# predict next day price 
x_train = []
y_train = []
for i in range(60,1258):                             # tere are total 1258 data 
    x_train.append(training_set_scaled[i-60:i,0])     # append first 60 data to x_train dataset 
    y_train.append(training_set_scaled[i,0])         #for first 60 x_train data 61st day data will be our y_train data  
    
x_train = np.array(x_train)
y_train  = np.array(y_train)



In [ ]:
x_train.shape           # our Training dataset

In [ ]:
y_train.shape          # our y training data

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))      #reshape your dataset with adding one extra dimension


## RNN Model Creation using Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()            # initiate Keras Sequential model object

In [ ]:
regressor.add(LSTM(units = 50,return_sequences = True,input_shape = (x_train.shape[1],1)))  #first layer input shape required 
regressor.add(Dropout(0.2))                               # setting dropout at each layer 
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))                             # output layer with one node for prediction
regressor.compile(optimizer = 'adam',loss = 'mean_squared_error')

In [ ]:
# Now our model is set now we need to our dataset
# fitting model with 100 epoch and batch size 32
# here we have given x_train and y_train , our model will first predict output from input dataset at each stage and again
# checked with y_label output then error will be calculated and furthaer weights assigned to nodes will be updated
regressor.fit(x_train,y_train,epochs = 100, batch_size = 32)

### Model Prediction

In [ ]:
# Load Test data 
dataset_test =pd.read_csv("google-stock-price-data\Google_Stock_Price_Test.csv")
real_stock_price = dataset_test.iloc[:,1:2].values


In [ ]:
# adding both test data and train data in a single dataframe for "Open" column
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis = 0)
dataset_total.head()

In [ ]:
#create input set for modele evaluation 
inputs = dataset_total[len(dataset_total) - len(dataset_test)-60:].values

inputs = inputs.reshape(-1,1)
inputs.shape                      # there are totale 80 data

In [ ]:
inputs = sc.transform(inputs)     # Scale input values 
inputs.shape


In [ ]:
# create x_test dataset as we have done earlier with 60 days grouped data 

x_test = []
for i in range(60,80):
    x_test.append(inputs[i-60:i,0])
    
x_test = np.array(x_test)
x_test.shape


In [ ]:
#  Reshape input dataset
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [ ]:
# Now predict stock price using predict method and passing x_test data
predicted_price = regressor.predict(x_test)
predicted_price

In [ ]:
predicted_price = sc.inverse_transform(predicted_price)   # get inverse_transformed value to map with original output
predicted_price

In [ ]:
predicted_price = predicted_price
plt.plot(real_stock_price,color = 'red', label = 'Real Price')
plt.plot(predicted_price, color = 'blue', label = 'Predicted Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

## We can see our Model is able to Recognise Upwards and Downwards of Prices 
## Our Blue line is able to Track than when price will go Up and when it will come down